# **Electrical Circuit Solver**

In [1]:
import numpy as np

A = np.random.randint(100,size = (10,10))
A = np.array(A,dtype = float)
b = np.random.randint(100,size =10)
b = np.array(b,dtype = float)

def Gauss(A,b):
    LenA = len(A)
    ans = [0]*LenA

    #Forward Elimination
    for i in range(LenA-1):
        if A[i][i] == 0:
            for j in range(i+1,LenA):
                if A[j,i] != 0:
                    # Swapping the rows 
                    A[[i,j]] = A[[j,i]]
                    b[[i,j]] = b[[j,i]]
                    break

        #Normalizing
        for m in range(i+1,LenA): A[i][m] = A[i][m]/A[i][i]
        b[i] = b[i]/A[i][i]
        A[i][i] = 1
    
        #Multiplication and elimination
        for j in range(i+1, LenA): 
            if A[j][i] != 0 : 
                frac = A[i][i]/A[j][i]
                for k in range(i,LenA):
                    A[j][k] = A[i][k] - A[j][k]*frac
                b[j] = b[i] - b[j]*frac

    if(A[LenA-1][LenA-1] == 0):
        if(b[LenA-1] !=0): print("No Solution")
        else: print("Infinite Solution")
        return
    #Back Substitution
    ans[LenA-1] = b[LenA-1]/A[LenA-1][LenA-1]
    for i in range(LenA-2,-1,-1):
        totsum =0
        for j in range(i+1,LenA):
            totsum +=  A[i][j]*ans[j]
        ans[i] = (b[i]-totsum)/A[i,i]

    #Return the answer
    return ans


print("Matrix A: ",A)
print("Matrix b: ",b)
print("\nSolution to the system Ax =b is (Without numpy): ", Gauss(A,b)) 
print("Time of execution: ", end ="")
%timeit Gauss(A,b)

print("\nSolution to the system Ax =b is (With numpy): ", np.linalg.solve(A,b)) 
print("Time of execution: ", end ="")
%timeit np.linalg.solve(A,b)    


Matrix A:  [[ 5. 62. 97. 26. 31. 54. 78. 21. 93. 16.]
 [ 1. 36. 37. 46. 70. 10. 43. 68. 43. 54.]
 [98. 87. 49.  9. 78. 28. 89. 86. 87. 46.]
 [75. 70. 34. 49. 76. 83. 89.  9. 54. 81.]
 [95.  6. 13. 99. 85.  0. 91. 71. 40. 82.]
 [53. 37. 86. 79. 17.  6. 92. 38. 90. 73.]
 [52. 18. 95. 27. 30. 76. 67. 13. 46. 27.]
 [20. 29. 12. 28. 88. 72. 30. 44. 25. 35.]
 [36. 15.  2. 49. 17. 83. 58. 16. 14.  3.]
 [55. 38. 16. 68. 71. 22. 41. 51. 57. 88.]]
Matrix b:  [20. 18. 36. 30. 83.  2. 23. 60.  5.  5.]

Solution to the system Ax =b is (Without numpy):  [-1.0489660910278502, -3.739393178382657, -1.6393439133585044, -2.8272888518592905, 0.8068968559139613, 0.48489273269250416, 2.5905494819305868, 0.006850520555579553, 2.3110889073275485, 1.329817391033592]
Time of execution: 56.5 µs ± 2.52 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Solution to the system Ax =b is (With numpy):  [-1.04896609 -3.73939318 -1.63934391 -2.82728885  0.80689686  0.48489273
  2.59054948  0.00685052  2.31108

- Matrix A is a random 10x10 Matrix and b is a random a radom 1x10 matrix. The solution matrix x for this system of linear equations have been obtained by using numpy.linalg.solve() and without that (Using gauss elimination). It turns out that the time of execution using numpy is greater than that of without using it. But as the size of the matrix A and b increase way much this trend is not seen, infact the inverse tread is seen that is numpy uses less time than gauss elimination in python. 
- Since only 10x10 system is asked to compare, I haven't used bigger system to show numpy uses less time (as it contains very optimised c/c++ code which is faster) as in question only 10x10 system is asked. Conclusion for 10x10 system is that numpy uses higher time than gauss elimation using python.

# Netlist

In [2]:
def Circuit_Solve(filename):
    
    samp1 = open(f"{filename}.txt",'r')
    line = samp1.readlines()
    flag,maxx,vcount,count,dc,aderror,nerror = 0,0,0,0,0,0,0
    simpleset = []
    #Removing unwanted data from the netlist
    for i in line:
        count +=1
        if(i.find(".end") != -1):
            break
        if (flag == 1):

            #Renaming GND to 0 to make all nodes as numbers
            i = i.replace("GND","0")
            for a in range(len(i)):
                if (i[a] == "#"): 
                    note = a
                    break
                else:note = 0
            if (note!=0): i = i[0:note]  
            listt = i.split() 

            #Removing the prefix n of any node in it's name if present 
            if (listt[1][0] == 'n'): listt[1] = listt[1][1:]
            if (listt[2][0] == 'n'): listt[2] = listt[2][1:]
            i = ' '.join(listt)
            simpleset.append(i.strip())

            #Getting the count of number of nodes excluding ground
            maxx = max(int(listt[1]),int(listt[2]),maxx)

            #Getting the count of number of independent voltage souces
            if (listt[0][0] == 'V') :vcount += 1 
            if(listt[3] == "dc"): dc =1
                
                

        if (i.find(".circuit") != -1):
            flag = 1
    samp1.close()

    line = line[count:]
    try:
        for l in line:
            lindex = line.index(l)
            if (l.split()[0]==".ac"):
                if(dc==1): 
                    #Checking the case where both ac and dc sources are present
                    aderror =1
                    break
                W = 2*np.pi*int(l.split()[2])
                if(lindex != 0):
                    if(W != W1): 
                        #Checking the case where multiple freqencies are present
                        nerror = 1
                        break
                W1 = W
            else:break
    except IndexError: pass

    print("\nRelevant part of the netlist\n", simpleset)

    #Defining two matrix of complex datatype
    A = np.zeros([(maxx+vcount),(maxx+vcount)],complex)
    b = np.zeros(maxx+vcount,complex)

    #Assigning values to matrix as per modified nodal analysis
    for i in simpleset:

    #Upadating the matrix as per MNA when resistance is encountered
        if(i[0]=='R'):
            if (int(i.split()[1]) != 0):
                    index = int(i.split()[1])-1
                    A[index][index] += 1/float(i.split()[3])
                
            if (int(i.split()[2]) != 0):
                    index = int(i.split()[2])-1
                    A[index][index] += 1/float(i.split()[3])

            if ((int(i.split()[1]) != 0) and (int(i.split()[2]) != 0)):
                A[int(i.split()[1])-1][int(i.split()[2])-1] += -1/float(i.split()[3])
                A[int(i.split()[2])-1][int(i.split()[1])-1] += -1/float(i.split()[3])
            

        #Upadating the matrix as per MNA when capacitor is encountered
        elif (i[0] == 'C'): 
            if (int(i.split()[1]) != 0):
                    index = int(i.split()[1])-1
                    A[index][index] += complex(0,float(i.split()[3])*W)      
            if (int(i.split()[2]) != 0):
                    index = int(i.split()[2])-1
                    A[index][index] += complex(0,float(i.split()[3])*W)
            if ((int(i.split()[1]) != 0) and (int(i.split()[2]) != 0)):
                A[int(i.split()[1])-1][int(i.split()[2])-1] += complex(0,-1*float(i.split()[3])*W)
                A[int(i.split()[2])-1][int(i.split()[1])-1] += complex(0,-1*float(i.split()[3])*W)


        #Upadating the matrix as per MNA when Inductor is encountered
        elif(i[0]=='L'):
            if (int(i.split()[1]) != 0):
                    index = int(i.split()[1])-1
                    A[index][index] += complex(0,-1/((float(i.split()[3]))*W))  
            if (int(i.split()[2]) != 0):
                    index = int(i.split()[2])-1
                    A[index][index] += complex(0,-1/((float(i.split()[3]))*W))
            if ((int(i.split()[1]) != 0) and (int(i.split()[2]) != 0)):
                A[int(i.split()[1])-1][int(i.split()[2])-1] += complex(0,1/((float(i.split()[3]))*W))
                A[int(i.split()[2])-1][int(i.split()[1])-1] += complex(0,1/((float(i.split()[3]))*W)) 


        #Upadating the matrix as per MNA when voltage source is encountered
        elif (i[0] == 'V'):
                line = maxx + int(i[1]) -1
                if(int(i.split()[1]) !=0):
                    A[line][int(i.split()[1])-1] = 1
                    A[int(i.split()[1])-1][line] = 1
                if(int(i.split()[2]) !=0):
                    A[line][int(i.split()[2])-1] = -1
                    A[int(i.split()[2])-1][line] = -1
                #For DC circuit
                if(i.split()[3] == "dc"): b[maxx+ int(i[1]) -1] = float(i.split()[4])
                #For AC circuit
                else: b[maxx+ int(i[1]) -1] = complex(float(i.split()[4])*np.cos(float(i.split()[5])), float(i.split()[4])*np.sin(float(i.split()[5])))


        #Upadating the matrix as per MNA when current source is encountered
        elif (i[0] == 'I'):
                #For AC Circuit
                if(i.split()[3] == "ac"):
                        if (int(i.split()[1]) !=0): b[int(i.split()[1])-1] += -1*complex(float(i.split()[4])*np.cos(float(i.split()[5])), float(i.split()[4])*np.sin(float(i.split()[5])))
                        if (int(i.split()[2]) !=0): b[int(i.split()[2])-1] +=  complex(float(i.split()[4])*np.cos(float(i.split()[5])), float(i.split()[4])*np.sin(float(i.split()[5])))
                #For DC Circuit
                else :
                        if (int(i.split()[1]) !=0): b[int(i.split()[1])-1] += -1*float(i.split()[4])
                        if (int(i.split()[2]) !=0): b[int(i.split()[2])-1] +=  float(i.split()[4])

        
    if(aderror == 0 and nerror == 0):
            print("\nThe matrix A: \n", A)
            print("\nThe matrix b: \n", b)
            #Overall solution
            Solution = Gauss(A,b)

            #Matrix containing nodal voltages
            Voltage = Solution[:maxx]

            #Matrix containing current through independant voltage source
            Current = Solution[maxx:]

            print("\n Nodal Voltages: ",Voltage)
            print("\n Current through independant sources: ", Current)
    elif ( aderror == 1): print("The circuit contains both ac and dc sources")
    elif (nerror == 1): print("The frequencies are different")


The function Circuit_Solve() aims to decode the netlist and convert them into matrix A and matrix b as per modified nodal analysis (MNA) and finally solve the system of equations using the previously defined Gauss function, that is it gives a matrix containg nodal voltages and another matrix containing current through indepedant sources.
<br>
<br>
I have taken care of two exception:
-  When both dc and ac sources are present
-  When more than one freqency exist in the circuit

In [3]:
Circuit_Solve("dc_circuit")


Relevant part of the netlist
 ['V1 0 1 dc 10', 'R1 1 2 1e3', 'R2 2 3 1e3', 'R3 3 4 1e3', 'R4 4 5 1e3', 'R5 2 0 2e3', 'R6 3 0 2e3', 'R7 4 0 2e3', 'R8 5 0 2e3']

The matrix A: 
 [[ 0.001 +0.j -0.001 +0.j  0.    +0.j  0.    +0.j  0.    +0.j -1.    +0.j]
 [-0.001 +0.j  0.0025+0.j -0.001 +0.j  0.    +0.j  0.    +0.j  0.    +0.j]
 [ 0.    +0.j -0.001 +0.j  0.0025+0.j -0.001 +0.j  0.    +0.j  0.    +0.j]
 [ 0.    +0.j  0.    +0.j -0.001 +0.j  0.0025+0.j -0.001 +0.j  0.    +0.j]
 [ 0.    +0.j  0.    +0.j  0.    +0.j -0.001 +0.j  0.0015+0.j  0.    +0.j]
 [-1.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j]]

The matrix b: 
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j 10.+0.j]

 Nodal Voltages:  [(-10.000000000000002+0j), (-5.029239766081872+0j), (-2.5730994152046787+0j), (-1.4035087719298247+0j), (-0.9356725146198831+0j)]

 Current through independant sources:  [(-0.00497076023391813-0j)]


The actual content in the file:

```There could be junk here!
Since this portion does not belong to circuit definition.
.circuit
V1 GND 1 dc 10    # There can be comment here
R1 1 2 1e3
R2 2 3 1e3
R3 3 4 1e3
R4 4 5 1e3      # comment can be in any line
R5 2 GND 2e3
R6 3 GND 2e3
R7 4 GND 2e3
R8 5 GND 2e3
.end
There could be junk here!
Since this portion does not belong to circuit definition.

The above is the output for a pure dc circuit. The actual netlist contains some unwanted stuffs, so in the output it is first refined and then matrix 
A, matrix b and then finally the matrix containing nodal voltages and the current through independant sources which are basically the solutions of the system are printed.

In [4]:
Circuit_Solve("ac_circuit")


Relevant part of the netlist
 ['V1 0 3 ac 5 0', 'C1 1 2 1', 'R1 2 3 1000', 'L1 1 0 1e-6']

The matrix A: 
 [[ 0.   +6124.03036409j  0.   -6283.18530718j  0.      +0.j
   0.      +0.j        ]
 [ 0.   -6283.18530718j  0.001+6283.18530718j -0.001   +0.j
   0.      +0.j        ]
 [ 0.      +0.j         -0.001   +0.j          0.001   +0.j
  -1.      +0.j        ]
 [ 0.      +0.j          0.      +0.j         -1.      +0.j
   0.      +0.j        ]]

The matrix b: 
 [0.+0.j 0.+0.j 0.+0.j 5.+0.j]

 Nodal Voltages:  [(-1.9239208801457085e-10-3.1415926534719644e-05j), (-1.8751873949430656e-10-3.062015181929001e-05j), (-5+0j)]

 Current through independant sources:  [(-0.004999999999812482+3.0620151819290015e-08j)]


The actual content in the file:

```.circuit
V1 GND n3 ac 5 0
C1 n1 n2 1
R1 n2 n3 1000
L1 n1 GND 1e-6
.end
.ac V1 1000

The above is the output for a pure ac circuit. The output format is similar to dc, the only differece is we need to take into account the impedance of capacitor and inductor and use phasor notation. The actual netlist contains some unwanted stuffs, so in the output it is first refined and then matrix 
A, matrix b and then finally the matrix containing nodal voltages and the current through independant sources which are basically the solutions of the system are printed.